In [ ]:
%matplotlib inline
import torch, torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
import re
import pickle
import collections
import bcolz
import pickle
from gensim.models import KeyedVectors
import numpy as np
import keras
from sklearn import model_selection


In [ ]:
path = '/run/media/backman/yay/giga-fren/'
fname = path + 'giga-fren.release2.fixed'
en_fname = fname + '.en/data'
fr_fname = fname + '.fr/data'

In [ ]:
re_eq = re.compile('^(Wh[^?.!]+\?)')
re_fq = re.compile('^([^?.!]+\?)')

In [ ]:
lines = ((re_eq.search(eq), re_fq.search(fq))
        for eq, fq in zip(open(en_fname), open(fr_fname)))

In [ ]:
qs = [(e.group(), f.group()) for e,f in lines if e and f]
len(qs)

In [ ]:
qs[:6]

In [ ]:
pickle.dump(qs, open(path+'qs.pkl', 'wb'))

In [ ]:
en_qs, fr_qs = zip(*qs)

In [ ]:
re_apos = re.compile(r"(\w)'s\b")         # make 's a separate word
re_mw_punc = re.compile(r"(\w[’'])(\w)")  # other ' in a word creates 2 words
re_punc = re.compile("([\"().,;:/_?!—])") # add spaces around punctuation
re_mult_space = re.compile(r"  *")        # replace multiple spaces with just one

def simple_toks(sent):
    sent = re_apos.sub(r"\1 's", sent)
    sent = re_mw_punc.sub(r"\1 \2", sent)
    sent = re_punc.sub(r" \1 ", sent).replace('-', ' ')
    sent = re_mult_space.sub(' ', sent)
    return sent.lower().split()

In [ ]:
fr_qtoks = list(map(simple_toks, fr_qs))

In [ ]:
en_qtoks = list(map(simple_toks, en_qs))

In [ ]:
PAD = 0
SOS = 1

def toks2ids(sents):
    voc_cnt = collections.Counter(t for sent in sents for t in sent)
    vocab = sorted(voc_cnt, key = voc_cnt.get, reverse = True)
    vocab.insert(PAD, "<PAD>")
    vocab.insert(SOS, "<SOS>")
    w2id = {w:i for i,w in enumerate(vocab)}
    ids = [[w2id[t] for t in sent] for sent in sents]
    return ids, vocab, w2id, voc_cnt

In [ ]:
fr_ids, fr_vocab, fr_w2id, fr_counts = toks2ids(fr_qtoks)
en_ids, en_vocab, en_w2id, en_counts = toks2ids(en_qtoks)

In [ ]:
glove_loc = '/run/media/backman/yay/glove/6B.100d'
en_vecs, en_wv_word, en_wv_idx = bcolz.open(glove_loc+'.dat')[:], pickle.load(open(glove_loc+'_words.pkl','rb'), encoding='latin1'), pickle.load(open(glove_loc+'_idx.pkl','rb'), encoding='latin1')
en_w2v = {w: en_vecs[en_wv_idx[w]] for w in en_wv_word}
n_en_vec, dim_en_vec = en_vecs.shape

In [ ]:
w2v_path = '/run/media/backman/yay/frWac_non_lem_no_postag_no_phrase_200_cbow_cut0.bin'
fr_model = KeyedVectors.load_word2vec_format(w2v_path, binary=True,unicode_errors='ignore')
ft_voc = fr_model.vocab
dim_fr_vec = 200

In [ ]:
def create_emb(w2v, targ_vocab, dim_vec):
    vocab_size = len(targ_vocab)
    emb = np.zeros((vocab_size, dim_vec))    
    found = 0
    
    for i, word in enumerate(targ_vocab):
        try: 
            emb[i] = w2v[word]
            found+=1
        except KeyError: emb[i] = np.random.normal(scale=0.6, size=(dim_vec,))
    return emb, found

In [ ]:
en_embds, found = create_emb(en_w2v, en_vocab, dim_en_vec)

In [ ]:
fr_embds, found = create_emb(fr_model, fr_vocab, dim_fr_vec)

In [ ]:
maxlen = 30
en_padded = pad_sequences(en_ids, maxlen, 'int64', 'post', 'post')
fr_padded = pad_sequences(fr_ids, maxlen, 'int64', 'post', 'post')

In [1]:
fr_train, fr_test, en_train, en_test = model_selection.train_test_split(
    fr_padded, en_padded, test_size=0.1)

NameError: name 'model_selection' is not defined